In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
CHROMOSOME = 22
PEAK_DIR = '/work-zfs/abattle4/heyuan/Variant_calling/datasets/GBR/ATAC_seq/alignment_bowtie/Peaks'
BAM_DIR = '/work-zfs/abattle4/heyuan/Variant_calling/datasets/GBR/ATAC_seq/alignment_bowtie/first_pass_bqsr'
Genotype_dir = '/work-zfs/abattle4/heyuan/Variant_calling/datasets/GBR/ATAC_seq/alignment_bowtie/Called_GT/'
VCF_dir = '/work-zfs/abattle4/heyuan/Variant_calling/datasets/GBR/ATAC_seq/alignment_bowtie/VCF_files'

In [ ]:
def read_in_peaks():

    # Peaks
    peaks = pd.read_csv('%s/peak_by_sample_matrix_chr%d.txt' % (PEAK_DIR, CHROMOSOME), sep=' ')
    peaks = peaks[[x for x in peaks.columns if 'Unnamed' not in x]]
    samples = [x for x in gt_dat.columns if x.startswith('HG')]

    # sample read depth
    bam_stats = pd.read_csv('%s/bam_stats.txt' % BAM_DIR, sep='\t', header=None)
    bam_stats.columns=['Sample', 'Reads']
    
    return [peaks, bam_stats, samples]

In [196]:
def compute_RPKM(peaks, bam_stats, samples):
    '''
    Compute RPKM (Reads per kilo base per million mapped reads)
    '''
    read_depth = np.array(bam_stats.set_index('Sample').loc[peaks.columns[4:]]['Reads'])
    peak_width = np.array(peaks['END'] - peaks['START']) / 1e6

    x = np.divide(np.array(peaks[peaks.columns[4:]]) , read_depth)
    RPKM = np.divide(x.transpose(), peak_width).transpose()
    RPKM_dat = pd.DataFrame(RPKM)

    RPKM_dat.columns = samples
    RPKM_dat['CHR'] = peaks['CHR']
    RPKM_dat['START'] = peaks['START']
    RPKM_dat['END'] = peaks['END']
    RPKM_dat['PEAK'] = peaks['PEAK']
    RPKM_dat = RPKM_dat[['PEAK', 'CHR', 'START', 'END']  + samples]
    RPKM_dat.to_csv('%s/peak_by_sample_matrix_chr%d_RPKM.txt' % (PEAK_DIR, CHROMOSOME), sep=' ', index = False)
    
    return RPKM_dat

In [198]:
RPKM_dat = compute_RPKM(peaks,bam_stats,samples)
RPKM_dat.head()

,PEAK,CHR,START,END,HG02215,HG00127,HG00155,HG00149,HG00243,HG00231,...,HG00105,HG00238,HG00245,HG00237,HG00116,HG00121,HG00239,HG00141,HG01789,HG00118
0,Peak197705,22,10673191,10673341,0.003427,0.003732,0.005241,0.003038,0.003430,0.004123,...,0.004714,0.003951,0.003671,0.008311,0.003139,0.004399,0.004156,0.003480,0.007316,0.003104
1,Peak197706,22,10678850,10679060,0.000874,0.000707,0.000643,0.000558,0.000507,0.000640,...,0.000935,0.000841,0.000234,0.001163,0.000766,0.000924,0.000614,0.000439,0.000871,0.000416
2,Peak197707,22,10687297,10687702,0.003052,0.002680,0.003791,0.003118,0.002015,0.002556,...,0.004074,0.002833,0.003521,0.003300,0.005303,0.003194,0.003769,0.002224,0.003402,0.002922
3,Peak197708,22,10713663,10713923,0.000989,0.000923,0.001559,0.001302,0.000887,0.001138,...,0.001511,0.000679,0.001399,0.001236,0.002209,0.000945,0.002067,0.000472,0.001454,0.001082
4,Peak197709,22,10714086,10714282,0.000250,0.001341,0.000439,0.000664,0.000543,0.000480,...,0.000802,0.002830,0.001104,0.001246,0.002168,0.000660,0.001316,0.000679,0.001244,0.000792


In [ ]:
def convert_gt_to_number(arri):
    '''
    convert genotype type A/A to 0, 1, 2 ...
    '''
    arri_idx = np.where(arri!='0')[0]
    nts = np.unique([a for b in [x.split('/') for x in arri[arri_idx]] for a in b])
    nts_dict = {}
    i = 0
    for ni in nts:
        nts_dict[ni] = i
        i += 1
    numeric_gt = [np.sum([nts_dict[a] for a in x.split('/')]) for x in arri[arri_idx]]
    numeric_gt_arr = np.ones(len(arri)) * (-1)
    numeric_gt_arr[arri_idx] = numeric_gt
    return numeric_gt_arr


def obtain_numerical_gt(gt_dat, samples):
    '''
    Very slow
    '''

    gt_numerical_dat = []
    for i in range(len(gt_dat)):
        arri = np.array(gt_dat[samples].iloc[i])
        token = convert_gt_to_number(arri)
        gt_numerical_dat.append(token)
        if i % 5000 == 0:
            print('%d rows finished' % i)
            
    gt_numerical_dat = pd.DataFrame(gt_numerical_dat)
    gt_numerical_dat.columns = samples
    gt_numerical_dat['CHR_POS'] = gt_numerical_dat['CHR_POS']
    gt_numerical_dat['CHR'] = gt_numerical_dat['CHR']
    gt_numerical_dat['POS'] = gt_numerical_dat['POS']
    gt_numerical_dat = gt_numerical_dat[['CHR_POS', 'CHR', 'POS']  + samples]
    
    gt_numerical_dat.to_csv('%s/gt_by_sample_matrix_chr%d_numericalGT.txt' % (Genotype_dir, CHROMOSOME), sep=' ', index = False)
    
    return gt_numerical_dat
    

In [403]:
def readin_genotype_info():
    ## Read in Genotpye
    gt_dat = pd.read_csv('%s/gt_by_sample_matrix_chr%d.txt' % (Genotype_dir, CHROMOSOME), sep=' ')
    gt_dat = gt_dat[[x for x in gt_dat.columns if 'Unnamed' not in x]]
    samples = [x for x in gt_dat.columns if x.startswith('HG')]

    # remove rows with less than 3 samples
    valid_snps = np.where(np.sum(np.array(gt_dat[samples]) != '0', axis=1) >= 3)[0]
    gt_dat = gt_dat.iloc[valid_snps].reset_index(drop=True)
    
    # remove rows with only one genotype
    validQTLsnps = np.where([len(set(x))>2 for x in np.array(gt_dat[samples])])[0]
    gt_dat = gt_dat.iloc[validQTLsnps].reset_index(drop=True)

    try:
        gt_numerical_dat = pd.read_csv('%s/gt_by_sample_matrix_chr%d_numericalGT.txt' % (Genotype_dir, CHROMOSOME), sep=' ')
    except:
        print('Convert genotype to numerical values')
        gt_numerical_dat = obtain_numerical_gt(gt_dat, samples)
        
    return gt_numerical_dat

In [404]:
def achieve_ll(rowi):
    '''
    Map PL scores back to likelihood
    '''
    
    pl_scores = [list(map(int, x.split(','))) for x in rowi]
    pl_scores = [np.array(x) for x in pl_scores]
    pl_scores = [x[x<10] for x in pl_scores]

    post_pp = [list(map(lambda x: 1/10 ** x, pl_s)) for pl_s in pl_scores]    
    max_post_pp = [np.max(x/np.sum(x)) for x in post_pp]
    
    return max_post_pp


def derive_ll(info_dat, samples):
    info_arr = np.array(info_dat[samples])
    idx_score = np.where(info_arr!='0')
    post_pp_arr = achieve_ll(info_arr[idx_score])

    post_pp = np.zeros(info_arr.shape)
    post_pp[idx_score] = post_pp_arr
    
    post_pp_dat = pd.DataFrame(post_pp)
    post_pp_dat.columns = samples
    post_pp_dat['CHR_POS'] = info_dat['CHR_POS']
    post_pp_dat['CHR'] = info_dat['CHR']
    post_pp_dat['POS'] = info_dat['POS']
    post_pp_dat = post_pp_dat[['CHR_POS', 'CHR', 'POS']  + samples]
    
    post_pp_dat.to_csv('%s/gt_infoLL_by_sample_matrix_chr%d.txt' % (VCF_dir, CHROMOSOME), sep=' ', index = False)
    
    return post_pp_dat

In [405]:
def readIn_genotype_info():
    ## Read in Genotpye INFO
    info_dat = pd.read_csv('%s/gt_info_by_sample_matrix_chr%d.txt' % (VCF_dir, CHROMOSOME), sep=' ')
    info_dat = info_dat[[x for x in info_dat.columns if 'Unnamed' not in x]]
    info_dat = info_dat.set_index('CHR_POS').loc[gt_dat['CHR_POS']].reset_index()
    
    samples = [x for x in info_dat.columns if x.startswith('HG')]
    
    try:
        post_pp_dat = pd.read_csv('%s/gt_infoLL_by_sample_matrix_chr%d.txt' % (VCF_dir, CHROMOSOME), sep=' ')
    except:
        print('Derive posterior probability for the genotypes')
        post_pp_dat = derive_ll(info_dat=info_dat, samples=samples)
        
    return post_pp_dat

In [200]:
#### Compute QTL

# Genotype data: gt_numerical_dat
# Peak data: RPKM_dat
# Genotype data weights: post_pp_dat

In [396]:
from statsmodels.regression import linear_model as sm

def compute_QTL_gti_peaki(gt_sample, peak_sample, weight_sample):
    #gt_sample = np.array(gt_numerical_dat[samples].iloc[0])
    #peak_sample = np.array(RPKM_dat[samples].iloc[0])
    #weight_sample = np.array(post_pp_dat[samples].iloc[0])
    valid_samples = np.where(gt_sample!= -1)[0]
    
    y = np.array(peak_sample[valid_samples])
    x = np.array(gt_sample[valid_samples])
    x_weights = np.array(weight_sample[valid_samples])
    
    wls_model = sm.WLS(y, x, weights = x_weights)
    results = wls_model.fit()
    
    return results.pvalues[0]

In [428]:
def compute_QTL_peaki(peaki, gt_numerical_dat, post_pp_dat):
    QTL_result_peaki = []
    #peaki = RPKM_dat[['START', 'END']].iloc[i]
    [start, end] = [peaki['START'], peaki['END']]
    
    SNPs_close = (np.array(gt_numerical_dat['POS']) < end + WINDOW) & (np.array(gt_numerical_dat['POS']) > start - WINDOW)
    SNPs_close = np.where(SNPs_close)[0]
    
    SNPs_gt = gt_numerical_dat.iloc[SNPs_close]
    SNPs_gt_ll = post_pp_dat.iloc[SNPs_close]
    
    for gti in range(len(SNPs_gt)):
        pval = compute_QTL_gti_peaki(SNPs_gt[samples].iloc[gti], peak_sample, SNPs_gt_ll[samples].iloc[gti])
        if(np.isnan(pval)):
            break
        snpid = SNPs_gt.iloc[gti]['CHR_POS']
        peakid = peaki['PEAK']
        QTL_result_peaki.append([peakid, snpid, pval])
        
    return QTL_result_peaki

In [429]:
QTL_results = []
for p in range(100):
    token = compute_QTL_peaki(RPKM_dat.iloc[p], gt_numerical_dat, post_pp_dat)
    QTL_results.append([a for b in token for a in b])
    if p % 100 == 0:
        print('%d peaks finished' % p)
        
QTL_results = [a for b in QTL_results for a in b]
QTL_results = np.reshape(np.array(QTL_results), [int(len(QTL_results)/3), 3])
QTL_dat = pd.DataFrame(QTL_results)
QTL_dat.columns = ['PeakID', 'CHR_POS', 'p-value']

0 peaks finished


In [433]:
RPKM_dat.iloc[90:110]

,PEAK,CHR,START,END,HG02215,HG00127,HG00155,HG00149,HG00243,HG00231,...,HG00105,HG00238,HG00245,HG00237,HG00116,HG00121,HG00239,HG00141,HG01789,HG00118
90,Peak197795,22,11702339,11703218,0.006336,0.004549,0.005785,0.001881,0.002462,0.013994,...,0.003143,0.003529,0.005482,0.013013,0.003136,0.012084,0.005344,0.009349,0.008753,0.003377
91,Peak197796,22,11703052,11703203,0.003972,0.003783,0.001871,0.002414,0.003290,0.004006,...,0.003209,0.002254,0.002800,0.003915,0.001901,0.004798,0.004057,0.005286,0.005814,0.002955
92,Peak197797,22,11706824,11707459,0.007093,0.010164,0.007390,0.009431,0.003800,0.011855,...,0.008723,0.009034,0.009338,0.015038,0.006384,0.008211,0.009191,0.009912,0.007527,0.010495
93,Peak197798,22,11706866,11706974,0.028559,0.044849,0.033552,0.047131,0.013306,0.052902,...,0.042679,0.044596,0.044252,0.071283,0.032220,0.038094,0.041102,0.044727,0.031839,0.052636
94,Peak197799,22,11814303,11814406,0.010219,0.010647,0.015027,0.010743,0.008957,0.010180,...,0.012713,0.009058,0.008784,0.025081,0.006355,0.014068,0.011687,0.012321,0.016692,0.020343
95,Peak197800,22,11814317,11815089,0.000698,0.000651,0.000859,0.000944,0.000437,0.001045,...,0.000780,0.000719,0.000802,0.000966,0.000372,0.000436,0.000780,0.000676,0.000648,0.000804
96,Peak197801,22,11814357,11814569,0.002483,0.002425,0.002260,0.002702,0.001339,0.002283,...,0.002718,0.002676,0.002644,0.002607,0.001517,0.001953,0.002281,0.002124,0.002243,0.003112
97,Peak197802,22,11815414,11815957,0.000496,0.000631,0.000317,0.000671,0.000294,0.000322,...,0.000555,0.000789,0.000688,0.000781,0.000402,0.000643,0.000574,0.000641,0.000539,0.000893
98,Peak197803,22,11815873,11815971,0.001374,0.001166,0.001880,0.003587,0.002172,0.001097,...,0.004142,0.002059,0.002810,0.001049,0.001289,0.002508,0.002194,0.003759,0.002737,0.003663
99,Peak197804,22,11816162,11816347,0.018194,0.015067,0.030078,0.020970,0.012562,0.018893,...,0.021447,0.016357,0.026418,0.024802,0.009808,0.027764,0.018766,0.020358,0.025244,0.016046


In [434]:
QTL_dat

,PeakID,CHR_POS,p-value
0,Peak197705,22_10663052,0.04235655122471262
1,Peak197705,22_10671024,0.15324283203019806
2,Peak197705,22_10690592,0.012848184127145985
3,Peak197705,22_10698566,0.3644675803804868
4,Peak197705,22_10716777,0.03783441924708301
5,Peak197705,22_10716849,1.2658853666858625e-07
6,Peak197705,22_10716902,0.006890333533298567
7,Peak197705,22_10717023,0.023175562097778216
8,Peak197705,22_10717060,1.7824568889002734e-15
9,Peak197705,22_10717387,5.763825147727e-07
